### モデルの再作製

本当に求めたいのは1度ずつの角度。

しかし、学習させるのは5度ずつの角度。

作製するモデルの出力は、theta 91, phi 91の合計8281....

これは流石に多すぎるのではないか。。。

以前に紹介した、分岐するネットワークの作成に挑戦する。

In [43]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Allocation GPU hardware ID = 0
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[1], 'GPU')

class MyModel(tf.keras.Model):
    
    def __init__(self):
        super().__init__()#これ何？
        self.inp_shape = (1, 61, 61, 150, 1)
        self.out_shape_1 = (91)
        self.out_shape_2 = (91)
        self.out_shape = (self.out_shape_1, self.out_shape_2)

        self.conv3d_1 = tf.keras.layers.Conv3D(16, 2, input_shape=self.inp_shape, activation = 'relu', strides = 2, name = "conv3d_1")
        self.batnorm_1 = tf.keras.layers.BatchNormalization()
        
        self.conv3d_2 = tf.keras.layers.Conv3D(32, 2, activation = 'relu', strides = 2, name = "conv3d_2")
        self.batnorm_2 = tf.keras.layers.BatchNormalization()
        
        self.conv3d_3 = tf.keras.layers.Conv3D(64, 2, activation = 'relu', strides = 2, name = "conv3d_3")

        self.flatten = tf.keras.layers.Flatten()

        self.dense_1 = tf.keras.layers.Dense(91, activation='softmax')
        self.dense_2 = tf.keras.layers.Dense(91, activation='softmax')

    def call(self, inputs):
        
        x = self.conv3d_1(inputs)
        x = self.batnorm_1(x)
        x = self.conv3d_2(x)
        x = self.batnorm_2(x)
        x = self.conv3d_3(x)
        x = self.flatten(x)
        x_1 = self.dense_1(x)
        x_2 = self.dense_2(x)
        
        return x_1, x_2
    


In [81]:
model = MyModel()
model.compile(loss      = tf.keras.losses.BinaryCrossentropy(),
              optimizer = {"theta":tf.keras.optimizers.SGD(0.2),
                           "phi"  :tf.keras.optimizers.SGD(0.2)}
             )
model.build(input_shape=model.inp_shape)
model.summary() 
model.count_params()

Model: "my_model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            multiple                  144       
_________________________________________________________________
batch_normalization_12 (Batc multiple                  64        
_________________________________________________________________
conv3d_2 (Conv3D)            multiple                  4128      
_________________________________________________________________
batch_normalization_13 (Batc multiple                  128       
_________________________________________________________________
conv3d_3 (Conv3D)            multiple                  16448     
_________________________________________________________________
flatten_6 (Flatten)          multiple                  0         
_________________________________________________________________
dense_12 (Dense)             multiple                  5

10294630

最後のdense二つは、並列になるはずなので、何かおかしい気がする。（改善ずみ）

In [82]:
x_train = np.load("data.npy")
x_train = np.expand_dims(x_train, axis=4)
t_train = np.load("theta.npy")
# t_train = np.expand_dims(t_train, axis=2)
p_train = np.load("phi.npy")
# p_train = np.expand_dims(p_train, axis=2)

x_train = np.array(x_train)
t_train = np.array(t_train)
p_train = np.array(p_train)

In [83]:
print(x_train.shape)
print(t_train.shape)
print(p_train.shape)

(361, 61, 61, 150, 1)
(361, 91)
(361, 91)


今回から使用する正解ラベルは1度刻みで[-45,45]の範囲とした。

試しに推論を行ってみる。推論の結果は。。。

In [84]:
pred = model.predict(x_train)

In [89]:
print(np.array(pred).shape)

print(type(x_train))
print(type(p_train))
print(type(t_train))

(2, 361, 91)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


ちゃんと二つ出力されている。

In [80]:
model.fit(x = x_train, y = {"theta":t_train, "phi":p_train})

TypeError: 'NoneType' object is not callable

うまくいかない。

諦めた。

Functional APIを使用する事でもっと簡単に実装できるらしいので、そちらで実行する。